# Open Brewery DB Notebook

In [155]:
import numpy as np
import pandas as pd
import datetime
from slugify import slugify

In [171]:
df = pd.read_csv('breweries.csv', header=0)

In [172]:
df.head()

,id,name,brewery_type,street,address_2,address_3,city,state,county_province,postal_code,website_url,phone,created_at,updated_at,country,longitude,latitude,tags
0,10-56-brewing-company-knox,10-56 Brewing Company,micro,400 Brown Cir,NaN,NaN,Knox,Indiana,NaN,46534,NaN,6308165790,2018-07-24 01:33:21,2018-08-24 00:32:25,United States,-86.627954,41.289715,NaN
1,10-barrel-brewing-co-bend,10 Barrel Brewing Co,large,62970 18th St,NaN,NaN,Bend,Oregon,NaN,97701-9847,http://www.10barrel.com,5415851007,2018-07-24 01:34:04,2018-08-11 21:39:09,United States,NaN,NaN,NaN
2,10-barrel-brewing-co-bend-2,10 Barrel Brewing Co,large,1135 NW Galveston Ave Ste B,NaN,NaN,Bend,Oregon,NaN,97703-2465,NaN,5415851007,2018-07-24 01:34:04,2018-08-11 21:39:09,United States,NaN,NaN,NaN
3,10-barrel-brewing-co-bend-pub-bend,10 Barrel Brewing Co - Bend Pub,large,62950 NE 18th St,NaN,NaN,Bend,Oregon,NaN,97701,NaN,5415851007,2018-07-24 01:34:04,2018-08-24 15:45:44,United States,-121.280954,44.091211,NaN
4,10-barrel-brewing-co-boise-boise,10 Barrel Brewing Co - Boise,large,826 W Bannock St,NaN,NaN,Boise,Idaho,NaN,83702-5857,http://www.10barrel.com,2083445870,2018-07-24 01:33:17,2020-10-11 17:00:13,United States,-116.202929,43.618516,NaN


In [173]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7797 entries, 0 to 7796
Data columns (total 18 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   id               7797 non-null   object 
 1   name             7797 non-null   object 
 2   brewery_type     7797 non-null   object 
 3   street           6974 non-null   object 
 4   address_2        8 non-null      object 
 5   address_3        2 non-null      object 
 6   city             7797 non-null   object 
 7   state            7787 non-null   object 
 8   county_province  10 non-null     object 
 9   postal_code      7797 non-null   object 
 10  website_url      6608 non-null   object 
 11  phone            6997 non-null   object 
 12  created_at       7797 non-null   object 
 13  updated_at       7797 non-null   object 
 14  country          7797 non-null   object 
 15  longitude        4932 non-null   float64
 16  latitude         4932 non-null   float64
 17  tags          

## Clean-up

In [164]:
# Get rid of rows without `country`
# Commenting these out because we might not need them anymore
df.dropna(subset=['country'], inplace=True)

In [123]:
# Fill empty `created_at` and `updated_at` values with current datetime
# Commenting these out because we might not need them anymore
# current_datetime = datetime.datetime.now().isoformat()
# df.created_at.fillna(current_datetime, inplace=True)
# df.updated_at.fillna(current_datetime, inplace=True)

In [124]:
# Let's fix any rows Excel destroyed.
# Commenting these out because we might not need them anymore
# df.loc[df.created_at.str.match('1900'),'created_at'] = '2018-07-24 01:34:37'
# df.loc[df.updated_at.str.match('1900'),'updated_at'] = '2018-07-24 01:34:37'

In [163]:
# Commenting these out because we might not need them anymore
df['city'] = df['city'].str.strip()

In [174]:
df['created_at'] = pd.to_datetime(df['created_at'])
df['updated_at'] = pd.to_datetime(df['updated_at'])

In [175]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7797 entries, 0 to 7796
Data columns (total 18 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           --------------  -----         
 0   id               7797 non-null   object        
 1   name             7797 non-null   object        
 2   brewery_type     7797 non-null   object        
 3   street           6974 non-null   object        
 4   address_2        8 non-null      object        
 5   address_3        2 non-null      object        
 6   city             7797 non-null   object        
 7   state            7787 non-null   object        
 8   county_province  10 non-null     object        
 9   postal_code      7797 non-null   object        
 10  website_url      6608 non-null   object        
 11  phone            6997 non-null   object        
 12  created_at       7797 non-null   datetime64[ns]
 13  updated_at       7797 non-null   datetime64[ns]
 14  country          7797 non-null   object 

### Duplicates

In [176]:
len(df['id'].unique())

7797

In [165]:
df['name_city'] = df['name'] + " " + df['city']
df['new_id'] = df['name_city'].apply(slugify)

In [166]:
# Iterate through duplicates and add incremental numbers. Ex. "10-barrel-bend-1" and "10-barrel-bend-2"

# Get all of the duplicates
df_dups = df[df.duplicated(['new_id'], keep=False)]

# For each duplicate row...
for _, row in df_dups.iterrows():
    print(f"Updating {row['new_id']}...")
    # Create a temporary dataframe of all of the duplicated rows
    df_temp = df[df['new_id'] == row['new_id']]
    
    # Loop over each of the same rows and update the `new_id` with sequential numbers
    count = 0
    for index, value in df_temp.iterrows():
        count += 1
        if count == 1: 
            print(f"{value['new_id']}")
            continue
        print(f"{value['new_id']}-{str(count)}")
        df.loc[index, 'new_id'] = f"{value['new_id']}-{str(count)}"

Updating peoria-artisan-brewery-peoria...
peoria-artisan-brewery-peoria
peoria-artisan-brewery-peoria-2
Updating peoria-artisan-brewery-peoria...
peoria-artisan-brewery-peoria
Updating brewery-in-planning-claremont-claremont...
brewery-in-planning-claremont-claremont
brewery-in-planning-claremont-claremont-2
Updating brewery-in-planning-claremont-claremont...
brewery-in-planning-claremont-claremont
Updating bootleggers-brewery-fullerton...
bootleggers-brewery-fullerton
bootleggers-brewery-fullerton-2
Updating bootleggers-brewery-fullerton...
bootleggers-brewery-fullerton
Updating highland-park-brewery-los-angeles...
highland-park-brewery-los-angeles
highland-park-brewery-los-angeles-2
Updating highland-park-brewery-los-angeles...
highland-park-brewery-los-angeles
Updating ballast-point-brewing-company-san-diego...
ballast-point-brewing-company-san-diego
ballast-point-brewing-company-san-diego-2
Updating ballast-point-brewing-company-san-diego...
ballast-point-brewing-company-san-diego


In [167]:
len(df['new_id'].unique())

8011

In [168]:
df['id'] = df['new_id']
df.drop(columns=['new_id', 'name_city'], inplace=True)

In [169]:
len(df['id'].unique())

8011

## Save CSV

In [170]:
df.to_csv('breweries.csv', index=False)